In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-text-classification1/data/train_5500.label.txt"
testDataPath = "s3://corpus-text-classification1/data/TREC_10.label.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            word = line.strip().split()
            label = word[0].split(":")[0]
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, cat_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [4]:
categories = ['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']
num_classes = len(categories)

cat_to_id = {'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)

testData = readfile(testDataPath)
trainData = readfile(trainDataPath)

print(len(testData),len(trainData))
trainData = np.r_[trainData,testData]
np.random.shuffle(trainData)
len(trainData)

seq_length = 37

Loading GloVe!
Completed!
400000 100 400000
500 5452


In [9]:
def train_split_data(model, train_data, categories):
    test_acc = []
    fold_id = 0

    tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, cat_to_id, num_classes, seq_length)
    print(len(tx),len(tx[0]),len(tx[1]))

    kf = KFold(n_splits=10)
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        test_acc.append(model_train(model,tx[train_i], ty[train_i],tx[test_i], ty[test_i],categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc


def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, model, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {model.inputX: x_batch1, model.inputY: y_batch1, model.dropoutKeepProb: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(model, x_train, y_train, x_val, y_val, categories):
    
    # save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
    flag = False

    for epoch in range(num_epochs):  # 20
        start_time = time.time()
        
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {model.inputX: x_batch, model.inputY: y_batch, model.dropoutKeepProb: dropout_keep_prob}
            session.run(model.trainOp, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.dropoutKeepProb] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                loss_val, acc_val = evaluate(session, model, x_val, y_val, model.loss, model.acc, 64)
                if acc_val > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_val
                    last_improved = total_batch
                    # saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''
                
                duration = time.time() - start_time
                output = 'Iter: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
                print(output.format(total_batch, loss_train, acc_train, loss_val, acc_val, duration, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                
                test_data_len = len(x_val)
                test_num_batch = int((test_data_len - 1) / batch_size) + 1

                y_test_cls = np.argmax(y_val, 1)  # 获得类别
                y_test_pred_cls = np.zeros(shape=len(x_val), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)

                for i in range(test_num_batch):  # 逐批次处理
                    start_id = i * batch_size
                    end_id = min((i + 1) * batch_size, test_data_len)
                    feed_dict = {
                        model.inputX: x_val[start_id:end_id],
                        model.dropoutKeepProb: 1.0
                    }
                    y_test_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

                # 评估
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                # 混淆矩阵
                print("Confusion Matrix...")
                cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                print(cm)
                
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return accuracy_score

In [6]:
# 构建adversarailLSTM模型
class AdversarailLSTM(object):

    def __init__(self, wordEmbedding):
        # 定义输入
        self.inputX = tf.placeholder(tf.int32, [None, seq_length], name='inputX')
        self.inputY = tf.placeholder(tf.int32, [None, num_classes], name='inputY')

        self.dropoutKeepProb = tf.placeholder(tf.float64, name='keep_prob')

        # 词嵌入层
        with tf.name_scope("wordEmbedding"):
            wordEmbedding = tf.Variable(initial_value=wordEmbedding)
            self.embeddedWords = tf.nn.embedding_lookup(wordEmbedding, self.inputX)

        # 计算softmax交叉熵损失
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                # self.y_pred_cls = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                self.y_pred_cls = tf.argmax(tf.nn.softmax(self.predictions),1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）
                # losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)

        
        with tf.name_scope("perturloss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                print("perturbSize:{}".format(perturWordEmbedding))
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                # perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)

        self.loss = loss + perturLoss
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(self.loss)
        # 将梯度应用到变量下，生成训练器
        self.trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.inputY, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        # self.loss = loss
        
        
    def _Bi_LSTMAttention(self, embeddedWords):
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向网络结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                    # 定义反向网络结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)
            # 采用动态rnn，可以动态地输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元组(output_fw, output_bw), 其中两个元素的维度都是[batch_size, max_time, hidden_size], fw和bw的hiddensize一样
            # self.current_state是最终的状态，二元组(state_fw, state_bw), state_fw=[batch_size, s], s是一个元组(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm,
                                                                          self.embeddedWords, dtype=tf.float64,
                                                                          scope="bi-lstm" + str(idx))

        # 在bi-lstm+attention论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到attention的输出
            output = self.attention(H)
            outputSize = hiddenSizes[-1]
            print("outputSize:{}".format(outputSize))

        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW", dtype=tf.float64,
                shape=[outputSize, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())

            outputB = tf.Variable(tf.constant(0.1, dtype=tf.float64, shape=[num_classes]), name="outputB")

            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")

            return predictions

    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层lstm神经元的数量
        hiddenSize = hiddenSizes[-1]

        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1, dtype=tf.float64))

        # 对bi-lstm的输出用激活函数做非线性转换
        M = tf.tanh(H)

        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size], 计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1], 每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))

        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, seq_length])

        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)

        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, seq_length, 1]))

        # 将三维压缩成二维sequeezeR = [batch_size, hissen_size]
        sequeezeR = tf.squeeze(r)

        sentenceRepren = tf.tanh(sequeezeR)

        # 对attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)

        return output

    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)

        var = tf.matmul(weights, powWordEmbedding)
        stddev = tf.sqrt(1e-6 + var)

        return (wordEmbedding - mean) / stddev

    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, epsilon)
        # print("perturbSize:{}".format(embedded+perturb))
        return embedded + perturb

    def _scaleL2(self, x, norm_length):
        # shape(x) = [batch, num_step, d]
        # divide x by max(abs(x)) for a numerically stable L2 norm
        # 2norm(x) = a * 2norm(x/a)
        # scale over the full sequence, dim(1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [7]:
hiddenSizes = [128]  # 定义LSTM的隐藏层（一层，128个神经元）
epsilon = 5

num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 50
batch_size = 64
print_per_batch = 30  # 每多少轮输出一次结果

lstm = AdversarailLSTM(embedding)

outputSize:128
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


perturbSize:Tensor("perturloss/Bi-LSTM/add_2:0", shape=(?, 37, 100), dtype=float64)
outputSize:128


In [ ]:
train_split_data(lstm,trainData,categories)

5952 37 37
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:  3.319, Train Acc: 20.31%, Val Loss:  3.299, Val Acc: 22.99%, Time: 21.86s *
Iter: 60, Train Loss:  3.122, Train Acc: 46.88%, Val Loss:  3.148, Val Acc: 41.11%, Time: 38.35s *
Epoch: 2
Iter: 90, Train Loss:  2.903, Train Acc: 46.88%, Val Loss:  2.825, Val Acc: 46.81%, Time: 54.32s *
Iter: 120, Train Loss:  2.664, Train Acc: 53.12%, Val Loss:  2.518, Val Acc: 54.19%, Time: 71.05s *
Iter: 150, Train Loss:  2.513, Train Acc: 45.31%, Val Loss:  2.399, Val Acc: 52.01%, Time: 87.54s 
Epoch: 3
Iter: 180, Train Loss:    1.8, Train Acc: 71.88%, Val Loss:  2.147, Val Acc: 59.23%, Time: 103.65s *
Iter: 210, Train Loss:  1.971, Train Acc: 73.44%, Val Loss:  2.023, Val Acc: 63.42%, Time: 120.11s *
Iter: 240, Train Loss:  2.387, Train Acc: 50.00%, Val Loss:  1.934, Val Acc: 64.09%, Time: 137.03s *
Epoch: 4
Iter: 270, Train Loss:  1.984, Train Acc: 57.81%, Val Loss:  1.801, Val Acc: 66.44%, Time: 153.46s *
Iter: 300, Train 

Epoch: 29
Iter: 2370, Train Loss: 0.2723, Train Acc: 93.75%, Val Loss:   1.76, Val Acc: 80.70%, Time: 1289.70s 
Iter: 2400, Train Loss: 0.3057, Train Acc: 92.19%, Val Loss:  1.906, Val Acc: 80.03%, Time: 1305.78s 
Iter: 2430, Train Loss: 0.2744, Train Acc: 95.31%, Val Loss:  1.975, Val Acc: 78.69%, Time: 1322.53s 
Epoch: 30
Iter: 2460, Train Loss: 0.1582, Train Acc: 96.88%, Val Loss:  1.803, Val Acc: 79.70%, Time: 1338.86s 
Iter: 2490, Train Loss: 0.1977, Train Acc: 96.88%, Val Loss:  1.961, Val Acc: 79.70%, Time: 1355.27s 
Iter: 2520, Train Loss:  0.341, Train Acc: 95.45%, Val Loss:  2.073, Val Acc: 77.68%, Time: 1371.66s 
Epoch: 31
Iter: 2550, Train Loss: 0.4086, Train Acc: 95.31%, Val Loss:  1.693, Val Acc: 80.70%, Time: 1388.14s 
Iter: 2580, Train Loss: 0.1636, Train Acc: 96.88%, Val Loss:  1.852, Val Acc: 80.70%, Time: 1404.70s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR   

Iter: 30, Train Loss:  3.274, Train Acc: 15.62%, Val Loss:  3.307, Val Acc: 20.67%, Time: 21.73s *
Iter: 60, Train Loss:  3.104, Train Acc: 35.94%, Val Loss:  3.024, Val Acc: 39.50%, Time: 38.04s *
Epoch: 2
Iter: 90, Train Loss:  2.708, Train Acc: 39.06%, Val Loss:  2.851, Val Acc: 41.34%, Time: 54.82s *
Iter: 120, Train Loss:  2.446, Train Acc: 50.00%, Val Loss:  2.701, Val Acc: 48.74%, Time: 71.15s *
Iter: 150, Train Loss:  2.384, Train Acc: 53.12%, Val Loss:  2.498, Val Acc: 53.45%, Time: 87.79s *
Epoch: 3
Iter: 180, Train Loss:  2.359, Train Acc: 50.00%, Val Loss:   2.31, Val Acc: 56.97%, Time: 104.78s *
Iter: 210, Train Loss:  2.298, Train Acc: 65.62%, Val Loss:  2.203, Val Acc: 60.00%, Time: 120.76s *
Iter: 240, Train Loss:  1.935, Train Acc: 68.75%, Val Loss:  2.029, Val Acc: 63.03%, Time: 132.35s *
Epoch: 4
Iter: 270, Train Loss:  1.655, Train Acc: 70.31%, Val Loss:  2.036, Val Acc: 61.34%, Time: 145.16s 
Iter: 300, Train Loss:  1.909, Train Acc: 65.62%, Val Loss:  1.774, Val A